In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import json
import folium

In [2]:
tree_dat = pd.read_csv( 'trees2015.csv' )#, dtype=str )
#tree_dat.info()

In [3]:
cols_4Qs = [ 'spc_common', 'boroname', 'health', 'steward', 'zipcode', 'zip_city', 'borocode', 'latitude', 'longitude' ]

tree_dat_4Qs = tree_dat[ cols_4Qs ]
print( tree_dat_4Qs.head() )
print( tree_dat_4Qs.info() )

        spc_common  boroname health steward  zipcode      zip_city  borocode  \
0        red maple    Queens   Fair    None    11375  Forest Hills         4   
1          pin oak    Queens   Fair    None    11357    Whitestone         4   
2      honeylocust  Brooklyn   Good    1or2    11211      Brooklyn         3   
3      honeylocust  Brooklyn   Good    None    11211      Brooklyn         3   
4  American linden  Brooklyn   Good    None    11215      Brooklyn         3   

    latitude  longitude  
0  40.723092 -73.844215  
1  40.794111 -73.818679  
2  40.717581 -73.936608  
3  40.713537 -73.934456  
4  40.666778 -73.975979  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   spc_common  652169 non-null  object 
 1   boroname    683788 non-null  object 
 2   health      652172 non-null  object 
 3   steward     652173 non-null  object 
 

In [4]:
print( set( tree_dat_4Qs['zipcode'] ) )
print( set( tree_dat_4Qs['spc_common'] ) )

{10280, 10281, 10282, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10312, 10314, 83, 11354, 11355, 11356, 11357, 11358, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429, 11430, 11432, 11433, 11434, 11435, 11436, 11451, 10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475, 10020, 11001, 11004, 11005, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10017, 10018, 10019, 11040, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10048, 10065, 10069, 11096, 10075, 11101, 11102, 11103, 11104, 11105, 11106, 1

In [5]:
decendingTrees = tree_dat_4Qs.groupby('spc_common').size().reset_index(name='counts').sort_values( 'counts' )

decendingTrees.iloc[30:60]

,spc_common,counts
66,cucumber magnolia,169
106,sassafras,171
13,Chinese chestnut,173
99,pond cypress,181
7,Amur cork tree,183
130,white pine,202
117,spruce,202
35,Oklahoma redbud,219
94,paperbark maple,220
126,two-winged silverbell,221


In [ ]:
# %% cleaning data

#exp = tree_dat_4Qs.head(20) 
print( set( tree_dat_4Qs['spc_common' ] ) )
print( set( tree_dat_4Qs['health' ] ) )
print( set( tree_dat_4Qs['steward' ] ) )

print( tree_dat_4Qs.info() )

# not all tree have info for columns we need species, health, steward
sorted_tree_dat = tree_dat_4Qs.sort_values( by=['spc_common'] )

msno.matrix( sorted_tree_dat )
msno.heatmap( sorted_tree_dat )

# Wow, OKay, how lucky: missingness is totally correlated. Will just drop rows with nan.
tree_dat_4Qs = tree_dat_4Qs.dropna()
tree_dat_4Qs.shape

In [6]:
NY_coords = [ 40.7128, -74.0060 ]
#CartoDB” (positron and dark_matter) “OpenStreetMap”
#“Stamen” (Terrain, Toner, and Watercolor)
ny_map = folium.Map( location = NY_coords, tiles = 'Stamen Watercolor', zoom_start = 10 )
ny_map

In [7]:
tree_dat_4Qs['health'].unique()

array(['Fair', 'Good', 'Poor', nan], dtype=object)

In [16]:
from folium.plugins import MarkerCluster
df_species = tree_dat_4Qs[ tree_dat_4Qs[ 'spc_common' ] == 'crepe myrtle' ]
def map_certain_trees( df_species ):
    health_class = [ 'Poor', 'Fair', 'Good' ]
    class_color = [ 'red', 'orange', 'green' ]
    marker_cluster = MarkerCluster().add_to( ny_map )
    for num,hclass in enumerate(health_class):
        df_species_class = df_species[ df_species[ 'health' ] == hclass ]
        for idx, row in df_species_class.iterrows():
            marker_poor = folium.Marker( location = [row['latitude'], row['longitude']],
                                        icon=folium.Icon(color=class_color[num], 
                                        icon='tree', prefix='fa')).add_to( marker_cluster )

In [17]:
map_certain_trees( df_species )

In [18]:
ny_map

In [192]:
# aggregate the data
boro_health_counts = df_species.groupby( ['boroname', 'health'] ).size().reset_index(name='counts_health')
boro_species_counts = df_species.groupby( ['boroname'] ).size().reset_index(name='counts_species')

In [193]:
boro_health_counts = boro_health_counts.merge( boro_species_counts, on = 'boroname' )
boro_health_counts['percent'] = boro_health_counts['counts_health']/boro_health_counts['counts_species']*100
# Create the dictionary
health_dictionary ={'Poor' : 1, 'Fair' : 2, 'Good' : 3}
color_dictionary ={'Poor' : 'gold', 'Fair' : 'greenyellow', 'Good' : 'forestgreen'}
boro_health_counts['rank_health'] = boro_health_counts['health'].map( health_dictionary )
boro_health_counts['color_health'] = boro_health_counts['health'].map( color_dictionary )
boro_health_counts = boro_health_counts.sort_values( by = ['boroname','rank_health'] ).reset_index()
boro_health_counts

,index,boroname,health,counts_health,counts_species,percent,rank_health,color_health
0,2,Bronx,Poor,5,45,11.111111,1,gold
1,0,Bronx,Fair,7,45,15.555556,2,greenyellow
2,1,Bronx,Good,33,45,73.333333,3,forestgreen
3,5,Brooklyn,Poor,3,65,4.615385,1,gold
4,3,Brooklyn,Fair,6,65,9.230769,2,greenyellow
5,4,Brooklyn,Good,56,65,86.153846,3,forestgreen
6,6,Manhattan,Good,5,5,100.000000,3,forestgreen
7,9,Queens,Poor,6,240,2.500000,1,gold
8,7,Queens,Fair,28,240,11.666667,2,greenyellow
9,8,Queens,Good,206,240,85.833333,3,forestgreen


Following a great example: [Drawing a Gauge with `matplotlib`](http://nicolasfauchereau.github.io/climatecode/posts/drawing-a-gauge-with-matplotlib/)

In [204]:
def needle_dir( boro_prop ):
    num_rows = boro_prop.shape[0]
    if num_rows == 3:
        ranges = [ [0, boro_prop[0]], 
             [boro_prop[0], boro_prop[0] + boro_prop[1]],
             [boro_prop[0] + boro_prop[1], boro_prop[0] + boro_prop[1] + boro_prop[2]]]
    elif num_rows == 2:
        ranges = [ [0, boro_prop[0]], 
             [boro_prop[0], boro_prop[0] + boro_prop[1]]]
    elif num_rows == 1:
        ranges = [ [0, boro_prop[0]]]
    maxidx = boro_prop.idxmax()
    needle_dir = sum( ranges[maxidx] )/2
    needle_dir = needle_dir*180/100 #rescale from 0-100 to 0-180
    needle_dir = 180-needle_dir #flip axis direction
    return needle_dir, maxidx

In [205]:
bronx_percent = boro_health_counts[ boro_health_counts['boroname']=='Bronx']
bronx_percent = bronx_percent['percent']
#bronx_percent.idxmax()
bronx_nd = needle_dir( bronx_percent )
bronx_nd

(66.0, 2)

In [206]:
import plotly.graph_objects as go


fig = go.Figure(go.Indicator(
    mode = "gauge",
    #value = 0,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Bronx", 'font': {'size': 36}},
    gauge = {
        'axis': {'range': [None, 100], 'tickwidth': 3, 'tickcolor': "gray", 'tickfont': { 'size':20}},
        'bgcolor': "white",
        'borderwidth': 4,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, boro_health_counts['percent'][0]], 'color': 'gold'},
            {'range': [boro_health_counts['percent'][0], boro_health_counts['percent'][0] + boro_health_counts['percent'][1]], 
             'color': 'greenyellow'},
            {'range': [boro_health_counts['percent'][0] + boro_health_counts['percent'][1], 
                        boro_health_counts['percent'][0] + boro_health_counts['percent'][1] + boro_health_counts['percent'][2]], 
             'color': 'forestgreen'}
        ],

    }))

fig.update_layout(paper_bgcolor = "cornsilk", font = {'color': "gray", 'family': "Arial"},
                     xaxis={'showgrid': False, 'range':[-1,1], 'showticklabels':False},
                     yaxis={'showgrid': False, 'range':[0,1], 'showticklabels':False},
                     plot_bgcolor='rgba(0,0,0,0)')


theta, maxidx = bronx_nd
r= 0.9
x_head = r * np.cos(np.radians(theta))
y_head = r * np.sin(np.radians(theta))

fig.add_annotation(
    ax=0,
    ay=0,
    axref='x',
    ayref='y',
    x=x_head,
    y=y_head,
    xref='x',
    yref='y',
    showarrow=True,
    arrowhead=3,
    arrowsize=2,
    arrowwidth=4,
    arrowcolor = 'gray'
    )

fig.show()

In [207]:
# functionalize the dial figure

def draw_boro_dial( boro_health_counts, boro ):
    boro_health_counts = boro_health_counts[ boro_health_counts['boroname'] == boro ].reset_index()
    num_ranks = boro_health_counts.shape[0]
    colors = boro_health_counts['color_health']
    if num_ranks == 3:
        steps_list = [
            {'range': [0, boro_health_counts['percent'][0]], 'color': colors[0]},
            {'range': [boro_health_counts['percent'][0], boro_health_counts['percent'][0] + boro_health_counts['percent'][1]], 
             'color': colors[1]},
            {'range': [boro_health_counts['percent'][0] + boro_health_counts['percent'][1], 
                        boro_health_counts['percent'][0] + boro_health_counts['percent'][1] + boro_health_counts['percent'][2]], 
             'color': colors[2]}]
    elif num_ranks == 2:
        steps_list = [
            {'range': [0, boro_health_counts['percent'][0]], 'color': colors[0]},
            {'range': [boro_health_counts['percent'][0], boro_health_counts['percent'][0] + boro_health_counts['percent'][1]], 
             'color': colors[1]}]
    elif num_ranks == 1:
        steps_list = [
            {'range': [0, boro_health_counts['percent'][0]], 'color': colors[0]}]
    fig = go.Figure(go.Indicator(
    mode = "gauge",
    #value = 0,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': boro, 'font': {'size': 36}},     
    gauge = {
        'axis': {'range': [None, 100], 'tickwidth': 3, 'tickcolor': "gray", 'tickfont': { 'size':20}},
        'bgcolor': "white",
        'borderwidth': 4,
        'bordercolor': "gray",
        'steps': steps_list,
    }))

    fig.update_layout(paper_bgcolor = "cornsilk", font = {'color': "gray", 'family': "Arial"},
                      xaxis={'showgrid': False, 'range':[-1,1], 'showticklabels':False},
                      yaxis={'showgrid': False, 'range':[0,1], 'showticklabels':False},
                      plot_bgcolor='rgba(0,0,0,0)')

    boro_percent = boro_health_counts['percent']
    boro_nd, maxidx = needle_dir( boro_percent )
    theta = boro_nd
    r= 0.6
    x_head = r * np.cos(np.radians(theta))
    y_head = r * np.sin(np.radians(theta))

    fig.add_annotation( ax=0, ay=0, axref='x', ayref='y', x=x_head, y=y_head,
                       xref='x', yref='y', showarrow=True, arrowhead=3, 
                       arrowsize=2, arrowwidth=4, arrowcolor = colors[maxidx] )
    
    return fig

In [208]:
draw_boro_dial( boro_health_counts, 'Queens' )

In [209]:
draw_boro_dial( boro_health_counts, 'Manhattan' )

In [213]:
boros = boro_health_counts['boroname'].unique()

for boro in boros:
    print( boro )

Bronx
Brooklyn
Manhattan
Queens
Staten Island


## Dash

In [6]:
import dash
import dash_bootstrap_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for your data.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 15:57:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 15:57:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 15:57:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 15:57:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 15:57:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


In [225]:
#controls
controls = dbc.FormGroup(
    [
        html.P('Dropdown', style={
            'textAlign': 'center'
        }),
        dcc.Dropdown(
            id='dropdown',
            options=[{
                'label': 'Value One',
                'value': 'value1'
            }, {
                'label': 'Value Two',
                'value': 'value2'
            },
                {
                    'label': 'Value Three',
                    'value': 'value3'
                }
            ],
            value=['value1'],  # default value
            multi=True
        ),
        html.Br(),
        html.P('Range Slider', style={
            'textAlign': 'center'
        }),
        dcc.RangeSlider(
            id='range_slider',
            min=0,
            max=20,
            step=0.5,
            value=[5, 15]
        ),
        html.P('Check Box', style={
            'textAlign': 'center'
        }),
        dbc.Card([dbc.Checklist(
            id='check_list',
            options=[{
                'label': 'Value One',
                'value': 'value1'
            },
                {
                    'label': 'Value Two',
                    'value': 'value2'
                },
                {
                    'label': 'Value Three',
                    'value': 'value3'
                }
            ],
            value=['value1', 'value2'],
            inline=True
        )]),
        html.Br(),
        html.P('Radio Items', style={
            'textAlign': 'center'
        }),
        dbc.Card([dbc.RadioItems(
            id='radio_items',
            options=[{
                'label': 'Value One',
                'value': 'value1'
            },
                {
                    'label': 'Value Two',
                    'value': 'value2'
                },
                {
                    'label': 'Value Three',
                    'value': 'value3'
                }
            ],
            value='value1',
            style={
                'margin': 'auto'
            }
        )]),
        html.Br(),
        dbc.Button(
            id='submit_button',
            n_clicks=0,
            children='Submit',
            color='primary',
            block=True
        ),
    ]
)

#style for sidebar
# the style arguments for the sidebar.
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '20%',
    'padding': '20px 10px',
    'background-color': '#f8f9fa'
}

# the style arguments for the main content page.
CONTENT_STYLE = {
    'margin-left': '25%',
    'margin-right': '5%',
    'top': 0,
    'padding': '20px 10px'
}

TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#191970'
}

CARD_TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#0074D9'
}

#define the sidebar
sidebar = html.Div(
    [
        html.H2('Parameters', style=TEXT_STYLE),
        html.Hr(),
        controls
    ],
    style=SIDEBAR_STYLE,
)

In [227]:
# content
content_first_row = dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        html.H4(id='card_title_1', children=['Card Title 1'], className='card-title',
                                style=CARD_TEXT_STYLE),
                        html.P(id='card_text_1', children=['Sample text.'], style=CARD_TEXT_STYLE),
                    ]
                )
            ]
        ),
        md=3
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        html.H4('Card Title 2', className='card-title', style=CARD_TEXT_STYLE),
                        html.P('Sample text.', style=CARD_TEXT_STYLE),
                    ]
                ),
            ]

        ),
        md=3
    ),
    dbc.Col(
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4('Card Title 3', className='card-title', style=CARD_TEXT_STYLE),
                        html.P('Sample text.', style=CARD_TEXT_STYLE),
                    ]
                ),
            ]

        ),
        md=3
    ),
    dbc.Col(
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4('Card Title 4', className='card-title', style=CARD_TEXT_STYLE),
                        html.P('Sample text.', style=CARD_TEXT_STYLE),
                    ]
                ),
            ]
        ),
        md=3
    )
])

content_second_row = dbc.Row(
    [
        dbc.Col(
            dcc.Graph(id='graph_1'), md=4
        ),
        dbc.Col(
            dcc.Graph(id='graph_2'), md=4
        ),
        dbc.Col(
            dcc.Graph(id='graph_3'), md=4
        )
    ]
)

content_third_row = dbc.Row(
    [
        dbc.Col(
            dcc.Graph(id='graph_4'), md=12,
        )
    ]
)

content_fourth_row = dbc.Row(
    [
        dbc.Col(
            dcc.Graph(id='graph_5'), md=6
        ),
        dbc.Col(
            dcc.Graph(id='graph_6'), md=6
        )
    ]
)

content = html.Div(
    [
        html.H2('Analytics Dashboard Template', style=TEXT_STYLE),
        html.Hr(),
        content_first_row,
        content_second_row,
        content_third_row,
        content_fourth_row
    ],
    style=CONTENT_STYLE
)

In [228]:
# initializing the app
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([sidebar, content])